In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
from datetime import datetime

In [ ]:
mol_2016 = pd.read_csv(r'C:\Users\alejandro.m.gizzi\Documents\Mine\Mol 2016.csv', sep=";")
mol_2017 = pd.read_csv(r'C:\Users\alejandro.m.gizzi\Documents\Mine\Mol 2017.csv', sep=";")
mol_2018 = pd.read_csv(r'C:\Users\alejandro.m.gizzi\Documents\Mine\Mol 2018.csv', sep=",")

In [ ]:
#DF formatting and consolidation

pd.set_option('float_format', '{:f}'.format)
mol_2017 = mol_2017.drop(['ID'],axis=1)
mol_2018.columns = mol_2017.columns


mol_2016['FECHA'] = pd.to_datetime(pd.Series(mol_2016['FECHA']), format="%d/%m/%Y")
mol_2017['FECHA'] = pd.to_datetime(pd.Series(mol_2017['FECHA']), format="%d/%m/%Y")
mol_2018['FECHA'] = pd.to_datetime(mol_2018['FECHA'])


mol_2016['Month'] = pd.DatetimeIndex(mol_2016['FECHA']).month
mol_2016['Year'] = pd.DatetimeIndex(mol_2016['FECHA']).year

mol_2017['Month'] = pd.DatetimeIndex(mol_2017['FECHA']).month
mol_2017['Year'] = pd.DatetimeIndex(mol_2017['FECHA']).year

mol_2018['Month'] = pd.DatetimeIndex(mol_2018['FECHA']).month
mol_2018['Year'] = pd.DatetimeIndex(mol_2018['FECHA']).year


ALLMOL = mol_2016.append(mol_2017, ignore_index=False)
ALLMOL = ALLMOL.append(mol_2018, ignore_index=False)

#creates column with weekday name
#https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
#https://webcache.googleusercontent.com/search?q=cache:GywFzb4hhjsJ:https://stackoverflow.com/questions/8380389/how-to-get-day-name-in-datetime-in-python+&cd=1&hl=es-419&ct=clnk&gl=ar
ALLMOL['WeekDay'] = ALLMOL['FECHA'].dt.strftime("%A")

#replace all missing values in total
#https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b
#ALLMOL.isnull().values.any()
ALLMOL['TOTAL'].fillna(0, inplace=True)
ALLMOL= ALLMOL[ALLMOL['LINEA'] !='TALLER/PRUEBAS']
ALLMOL.dropna(how='any', inplace=True)

ALLMOL.to_csv("ALLMOL.csv", sep=';', encoding='utf-8')


In [ ]:
#-------------------------trash-----------------------------------------------------|
#mol_2018.rename(columns={'fecha': 'FECHA'}, inplace=True)
#mol_2018.rename(columns={'total': 'TOTAL'}, inplace=True)
#mol_2018['fecha'] = pd.to_datetime(pd.Series(mol_2018['fecha']), format="%d/%m/%Y")|
#mol_2016['Mes'] = mol_2016['PERIODO']                                              |
#mol_2016['Mes'] = mol_2016['FECHA'].Month                                          |
#mol_2016['Mes'] = mol_2016['FECHA'].Year                                           |
#df['Mes'] = df['Mes'].str[-2:]                                                     |
#mol_2016_Totpagos = mol_2016.set_index('Month', inplace=False)                     |
#mol_2016_Totpagos = mol_2016.groupby(level=0)['TOTAL'].sum()                       |
#mol_2016_Totpagos                                                                  |
#-------------------------trash-----------------------------------------------------|



In [ ]:
#DF totals by period
TotPag16 = mol_2016
TotPag16.set_index(["Year","Month"], inplace=True)
TotPag16 = TotPag16.sum(level=["Year","Month"])
mol_2016 = mol_2016.reset_index()

TotPag17 = mol_2017
TotPag17.set_index(["Year","Month"], inplace=True)
TotPag17 = TotPag17.sum(level=["Year","Month"])
mol_2017 = mol_2017.reset_index()

TotPag18 = mol_2018
TotPag18.set_index(["Year","Month"], inplace=True)
TotPag18 = TotPag18.sum(level=["Year","Month"])
mol_2018 = mol_2018.reset_index()

TotPaghist = TotPag16.append(TotPag17, ignore_index=False)
TotPaghist = TotPaghist.append(TotPag18, ignore_index=False)
TotPaghist = TotPaghist.drop(['PERIODO','PAX_PAGOS','PAX_PASES_PAGOS','PAX_FRANQ'],axis=1)


In [ ]:
#line plot
#fig, ax = plt.subplots(figsize=(50,5))
#ax.plot(TotPaghist['Year'].map(str) + " " + TotPaghist['Month'].map(str), TotPaghist['TOTAL'], color='r')
#plt.show()


#Scatter plot
#fig, ax = plt.subplots(figsize=(50,5))
#ax.scatter(TotPaghist['Year'].map(str) + " " + TotPaghist['Month'].map(str), TotPaghist['TOTAL'], edgecolors='r')
#plt.show()


#plot
#fig, ax = plt.subplots(figsize=(50,5))
#ax.plot_date(ALLMOL['FECHA'], ALLMOL['TOTAL'], color='r')
#plt.show()


#interactive plot kills ram
#data = [go.Scatter(x=ALLMOL['FECHA'], y=ALLMOL['TOTAL'])]

#ply.iplot(data, filename = 'time-series-simple')

In [ ]:
#DF to see std and mean for each day

#https://webcache.googleusercontent.com/search?q=cache:iBCzF1giI64J:https://stackoverflow.com/questions/30482071/how-to-calculate-mean-values-grouped-on-another-column-in-pandas+&cd=1&hl=es-419&ct=clnk&gl=ar
WeekdayTotalmean = ALLMOL.groupby('WeekDay', as_index=False)['TOTAL'].mean()

#https://webcache.googleusercontent.com/search?q=cache:cHr8bCdXsYUJ:https://stackoverflow.com/questions/20350863/pandas-dataframe-groupby-std+&cd=1&hl=es-419&ct=clnk&gl=ar
WeekdayTotalstd = ALLMOL.groupby('WeekDay')['TOTAL'].std()

#https://webcache.googleusercontent.com/search?q=cache:oEh9T7lHF9EJ:https://stackoverflow.com/questions/37968785/merging-two-dataframes+&cd=1&hl=es-419&ct=clnk&gl=ar
WDTmeanvar = pd.merge(WeekdayTotalmean,WeekdayTotalstd.to_frame(),how='left',on=['WeekDay'])
WDTmeanvar.rename(columns={'TOTAL_x': 'MEAN','TOTAL_y': 'STD'}, inplace=True)
WDTmeanvar['+/- Dif'] =  abs(WDTmeanvar['MEAN'] - WDTmeanvar['STD']) 
WDTmeanvar

In [3]:
ALLMOL = pd.read_csv(r'C:\Users\alejandro.m.gizzi\Documents\Mine\ALLMOL.csv', sep=";")


(35482585, 15)

array(['LINEA_A', 'LINEA_B', 'LINEA_C', 'LINEA_D', 'LINEA_E', 'LINEA_H',
       'TALLER/PRUEBAS', 'LineaA', 'LineaB', 'LineaC', 'LineaD', 'LineaE',
       'LineaH'], dtype=object)

(35482529, 15)